# Probability distribution using Oscilloscope

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

# Load the CSV file (replace with your actual file path)
file_path = 'tek0006ALL.csv'
df = pd.read_csv(file_path, skiprows=20)

# Rename columns
df.columns = ['TIME', 'CH2', 'CH3', 'CH4']

# Extract time and signal data (assuming CH4 as example)
time = df['TIME']
ch4 = df['CH4']

# Define the interval width
interval_width = 5.039999999983946e-08  # Time interval width

# Define photon ranges (in volts)
photon_ranges = {
    '1 Photon': (-0.72, -0.43),
    '2 Photon': (-1.13, -0.73),
    '3 Photon': (-1.51, -1.14),
    '4 Photon': (-1.95, -1.52),
}

# Classify peaks
classified_peaks = {'0 Photon': [],'1 Photon': [], '2 Photon': [], '3 Photon': [], '4 Photon': []}

# Create bins based on the interval width
time_min, time_max = np.min(time), np.max(time)
bins = np.arange(time_min, time_max, interval_width)

# Find peaks in the signal
peaks, _ = find_peaks(-ch4, height=0.35)

# Initialize lists to store results
classified_intervals = []

# Iterate over each interval
for i in range(len(bins) - 1):
    # Define the start and end of the interval
    start, end = bins[i], bins[i + 1]

    # Get indices of peaks within this interval
    peaks_in_interval = peaks[(time[peaks] >= start) & (time[peaks] < end)]

    # Initialize classification for the interval
    interval_classification = {'Interval Start': start, 'Interval End': end, 'Classification': '0 Photons'}

    # Check if there are peaks in the interval
    if len(peaks_in_interval) > 0:
        # Find the highest peak in the interval
        max_peak_idx = peaks_in_interval[np.argmax(ch4.iloc[peaks_in_interval])]
        max_amplitude = ch4.iloc[max_peak_idx]

        # Classify the peak based on photon ranges
        for photon, (lower, upper) in photon_ranges.items():
            if lower <= max_amplitude <= upper:
                interval_classification['Classification'] = photon
                classified_peaks[photon].append(max_peak_idx)
                break  # Stop checking after classification
    else:
        # If no peaks, classify as 0 photons
        interval_classification['Classification'] = '0 Photons'
        classified_peaks['0 Photon'].append(0)

    # Append the interval classification
    classified_intervals.append(interval_classification)


# Convert results to a DataFrame for better visualization
classified_intervals_df = pd.DataFrame(classified_intervals)

# Display results
print(classified_intervals_df)

# Optional: Plot the signal with classified peaks
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(time, ch4, color='blue', alpha=0.7, label='Signal')


# Plot peaks for each photon category
colors = ['gray', 'green', 'orange', 'red', 'purple']
for i, (photon, peak_indices) in enumerate(classified_peaks.items()):
    plt.scatter(time.iloc[peak_indices], ch4.iloc[peak_indices],
                label=f'{photon} Peaks', color=colors[i])

# Plot settings
plt.title('Classified Peaks in Intervals', fontsize=16)
plt.xlabel('Time (s)', fontsize=14)
plt.ylabel('Amplitude (V)', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import poisson

# Generate histogram data
photon_counts = {photon: len(indices) for photon, indices in classified_peaks.items()}

# Convert the dictionary to numeric values
categories = np.array([0, 1, 2, 3, 4])  # Numeric photon categories
counts = np.array(list(photon_counts.values())) # Observed counts
total_count = np.sum(counts)  # Total observed count
probabilities = counts/total_count

# Define a function for Poisson fitting
def poisson_scaled(k, lamb):
    """Poisson PMF scaled to match the observed total count."""
    return poisson.pmf(k, lamb)

# Fit the Poisson distribution to the observed counts
popt, _ = curve_fit(poisson_scaled, categories, probabilities , p0=[1])  # Initial guess for lambda=1
fitted_lambda = popt[0]

# Generate the fitted Poisson counts
poisson_fit_counts = poisson_scaled(categories, fitted_lambda)

# Plot the histogram and Poisson fit
plt.figure(figsize=(10, 6))

# Observed counts
plt.bar(categories - 0.2, probabilities , width=0.4, label='Observed', color='blue', alpha=0.7)

# Poisson fit
plt.bar(categories + 0.2, poisson_fit_counts, width=0.4, label='Poisson Fit', color='orange', alpha=0.7)

# Customize the plot
plt.title('Photon Classification and Poisson Fit', fontsize=16)
plt.xlabel('Photon Category', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(categories, [f'{c} Photons' for c in categories])
plt.legend(fontsize=12)
plt.grid(axis='y', alpha=0.5)

# Show the plot
plt.show()

# Print the fitted lambda
print(f"Fitted lambda (mean photon count): {fitted_lambda:.3f}")


NameError: name 'classified_peaks' is not defined

# Getting the $P$ matrix

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.special import factorial


P = np.array([
    [1, 0.076, 0.0063, 0, 0, 0, 0, 0, 0, 0],
    [0, 0.924, 0.507, 0.260, 0.135, 0.0716, 0.0383, 0.0207, 0.0113, 0.0062],
    [0, 0, 0.487, 0.647, 0.6728, 0.6482, 0.6067, 0.5596, 0.5139, 0.4712],
    [0, 0, 0, 0.0919, 0.1858, 0.2645, 0.3275, 0.3777, 0.4177, 0.4498],
    [0, 0, 0, 0, 0.00588, 0.0157, 0.0281, 0.0419, 0.0569, 0.0767]
])

def poisson_distribution(P, tau, wavelength, max_m):
    """
    Generate a Poisson distribution array.

    Parameters:
    - P: Laser power (in watts).
    - tau: Pulse duration (in seconds).
    - wavelength: Laser wavelength (in meters).
    - max_m: Maximum number of photons to compute in the distribution.

    Returns:
    - Array of probabilities for photon numbers 0 to max_m.
    """
    # Constants
    h = 6.626e-34  # Planck's constant (J·s)
    c = 3e8        # Speed of light (m/s)

    # Calculate the frequency of the laser
    nu = c / wavelength

    # Calculate the mean photon number (mu)
    mu = (P * tau) / (h * nu)
    # Generate the Poisson distribution
    m_values = np.arange(0, max_m)  # Photon numbers from 0 to max_m
    S_m = (mu ** m_values / factorial(m_values)) * np.exp(-mu)

    return S_m

def euclidean_norm(P_flat, Q, S, n, m):
    """
    Compute the Euclidean norm ||Q - PS||_2.

    Parameters:
    - P_flat: Flattened lower triangular matrix (optimization variable).
    - Q: Observed statistics vector (numpy array).
    - S: Input statistics vector (numpy array).
    - n: Number of pixels.
    - m: Maximum number of photons.

    Returns:
    - Euclidean norm ||Q - PS||_2.
    """
    # Reconstruct the upper triangular P matrix
    P_z = np.zeros((n, m))
    triu_indices = np.triu_indices(n, 0, m)
    P_z[triu_indices] = P_flat
    # Compute PS
    PS = np.dot(P_z, S)
    
    # Compute the Euclidean norm
    norm = np.linalg.norm(Q - PS, ord=2)
    return norm

def optimize_P(Q, S, n, m):
    """
    Optimize the detector response matrix P to minimize ||Q - PS||_2.

    Parameters:
    - Q: Observed statistics vector (numpy array).
    - S: Input statistics vector (numpy array).
    - n: Number of pixels.
    - m: Maximum number of photons.

    Returns:
    - Optimized detector response matrix P.
    """
    # Extract indices for the upper triangular part of the matrix
    triu_indices = np.triu_indices(n, 0, m)
    
    # Initial guess for P (flat array of upper triangular elements)
    P_initial = P[triu_indices]
    
    # Adjust bounds to match the size of the upper triangular part
    num_params = len(P_initial)
    bounds = [(0, 1) for _ in range(num_params)]
    
    # Optimization
    result = minimize(
        euclidean_norm, P_initial, args=(Q, S, n, m),
        method='L-BFGS-B', bounds=bounds
    )
    
    # Reconstruct the optimized upper triangular P matrix
    P_optimized = np.zeros((n, m))
    P_optimized[triu_indices] = result.x
    return P_optimized


# Example usage
p = 255e-9  # Laser peak power in watts 
tau = 0.3e-12  # Pulse duration in seconds 
wavelength = 1310e-9  # Laser wavelength in meters

# Example usage
n = 5  # Number of pixels (0 to 3 clicks)
m = 10  # Maximum number of photons considered (0 to 4 photons)

# Observed statistics Q (example)
Q = probabilities

# Input statistics S (Poisson distribution with mean photon number)
S = poisson_distribution(p, tau, wavelength, m)
print("Poisson Distribution:", S)

# Optimize P
P_optimized = optimize_P(Q, S, n, m)
print("Optimized P:\n", P_optimized)

#triu_indices = np.triu_indices(5, 0, 10)
#p_T = P[triu_indices]

#eu = euclidean_norm(p_T.flatten(), Q, S, n, m)
#print(eu)


Poisson Distribution: [6.04018582e-01 3.04516159e-01 7.67609591e-02 1.28996873e-02
 1.62584536e-03 1.63934091e-04 1.37745706e-05 9.92064878e-07
 6.25187278e-08 3.50209294e-09]
Optimized P:
 [[1.49364079e-01 0.00000000e+00 1.26963120e-01 3.16182877e-03
  4.03663831e-04 1.28385913e-05 2.71116391e-05 7.95439225e-06
  1.01271011e-06 4.81186598e-06]
 [0.00000000e+00 5.42173978e-01 4.10790300e-01 2.43737770e-01
  1.33018814e-01 7.14118710e-02 3.82652137e-02 2.07002972e-02
  1.13020675e-02 6.20000000e-03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 6.15751803e-01
  5.17350776e-01 4.71279329e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 7.97473616e-01 4.11560165e-01
  4.19950210e-01 4.50072012e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e+00 1.00000000e+00 2.78510889e-01 5.97590016e-02
  5.79975279e-02 7.67333744e-02]]
